In [1]:
%pip install torch
%pip install transformers

import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM
import logging
import pandas as pd
import numpy as np
logging.basicConfig(level=logging.INFO)# OPTIONAL

properties_df = pd.read_csv('../../data/experiment2/bert_large_property_probabilities_blank.csv')  
# properties_df = pd.read_csv('../../data/experiment2/bert_base_property_probabilities_blank.csv') # uncomment to run bert-base
properties_df = properties_df.fillna("")
properties_df


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


,model,property,category,question,answer,probability
0,bert_large,purifies water,purpose,An animal that,,
1,bert_large,enables decomposition,purpose,An animal that,,
2,bert_large,makes honey,purpose,An animal that,,
3,bert_large,aerates soil,purpose,An animal that,,
4,bert_large,pointy ears,biological,An animal that,,
5,bert_large,long legs,biological,An animal that,,
6,bert_large,hair,biological,An animal that,,
7,bert_large,warm blooded,biological,An animal that,,
8,bert_large,jump,behavior,An animal that,,
9,bert_large,swim,behavior,An animal that,,


In [2]:


tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
model = BertForMaskedLM.from_pretrained('bert-large-uncased')
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') # uncomment to run bert-base
# model = BertForMaskedLM.from_pretrained('bert-base-uncased') # uncomment to run bert-base
model.eval()
# model.to('cuda')  # if you have gpu

def predict_masked_sent(text, top_k=5):

    '''
    input: text, top_k = how many top probable words
    output: tuple ([str(word)], [float(weight)])
    '''
    # Tokenize input
    tokenized_text = tokenizer.tokenize(tokenizer.cls_token + text + tokenizer.sep_token)
    masked_index = tokenized_text.index("[MASK]")
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    # tokens_tensor = tokens_tensor.to('cuda')    # if you have gpu

    # Predict all tokens
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    probs = torch.nn.functional.softmax(predictions[0, masked_index], dim=-1)
    top_k_weights, top_k_indices = torch.topk(probs, top_k, sorted=True)

    token = []
    weight = []
    for i, pred_idx in enumerate(top_k_indices):
        predicted_token = tokenizer.convert_ids_to_tokens([pred_idx])
        token_weight = top_k_weights[i]
        token.append(predicted_token)
        weight.append(token_weight.tolist())

    return token, weight
        #print("[MASK]: '%s'"%predicted_token, " | weights:", float(token_weight))

Downloading: 100%|██████████| 226k/226k [00:00<00:00, 3.38MB/s]
Downloading: 100%|██████████| 28.0/28.0 [00:00<00:00, 11.1kB/s]
Downloading: 100%|██████████| 571/571 [00:00<00:00, 361kB/s]
Downloading: 100%|██████████| 1.25G/1.25G [00:37<00:00, 35.7MB/s]
Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
result_rows = []

for i, row in properties_df.iterrows():
    prompt = f"{row['question']} {row['property']} is a [MASK]."
    answers, probabilities = predict_masked_sent(prompt, top_k=5)
 

    # Loop through each answer and create a new row with its corresponding probability
    for j in range(len(answers)):
        result_row = {
            'model': row['model'],
            'property': row['property'],
            'category': row['category'],
            'question': row['question'],
            'answer': answers[j][0],
            'probability': probabilities[j]
        }
       # Append the result row to the list of result rows
        result_rows.append(result_row)

result_df = pd.DataFrame(result_rows)

print(result_df)

# prompt = "An animal that .... is a [MASK]."
# predict_masked_sent(prompt)

# answers = [item for sublist in answers for item in sublist]
    # probabilities = [str(p) for p in probabilities]
    # properties_df.at[i, 'answer'] = ', '.join(answers)
    # properties_df.at[i, 'probability'] = ', '.join(probabilities)

         model                       property category         question  \
0   bert_large                 purifies water  purpose  An animal that    
1   bert_large                 purifies water  purpose  An animal that    
2   bert_large                 purifies water  purpose  An animal that    
3   bert_large                 purifies water  purpose  An animal that    
4   bert_large                 purifies water  purpose  An animal that    
..         ...                            ...      ...              ...   
75  bert_large  share food with group members   social  An animal that    
76  bert_large  share food with group members   social  An animal that    
77  bert_large  share food with group members   social  An animal that    
78  bert_large  share food with group members   social  An animal that    
79  bert_large  share food with group members   social  An animal that    

      answer  probability  
0       frog     0.146590  
1       fish     0.087730  
2        pig   

In [4]:
result_df.to_csv('../../data/experiment2/bert_large_property_probabilities.csv', index=False)
# result_df.to_csv('../../data/experiment2/bert_base_property_probabilities.csv', index=False) # uncomment to run bert-base